[Info_pd_np_plt_seaborn_sklearn](http://localhost:8888/notebooks/PycharmProjects/ML%20learning/stepik%20%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B2%20Data%20Science%20%D0%B8%20%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5%20%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5/Info_pd_np_plt_seaborn_sklearn.ipynb)




[MetaTrader module for integration with Python](https://www.mql5.com/en/docs/integration/python_metatrader5)

In [1]:
# sklearn
# train_test_split Разделите массивы или матрицы на случайные поезда и тестовые подмножества.

# RandomForestClassifier — это метаоценка, которая соответствует ряду классификаторов дерева 
# решений для различных подвыборок набора данных и использует усреднение для повышения 
# точности прогнозирования и контроля переобучения.

# train_test_split Разделите массивы или матрицы на случайные тренировоные и тестовые подмножества.

# GridSearchCV реализует метод «подгонки» и «оценки».
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble        import RandomForestClassifier

# precision_score — это интуитивно способность классификатора не маркировать отрицательный 
# образец как положительный.tp / (tp + fp)tpfp

# recall_score это отношение, где - количество истинных положительных результатов и количество 
# ложных отрицательных результатов. Под отзывом понимается интуитивно способность классификатора 
# находить все положительные образцы.tp / (tp + fn)tpfn

# В классификации с несколькими метками эта функция вычисляет точность подмножества.

from sklearn.metrics         import precision_score, recall_score, accuracy_score

# confusion_matrix - Вычислите матрицу путаницы, чтобы оценить точность классификации.

# ConfusionMatrixDisplay - Визуализация матрицы путаницы.

# r2_score - (коэффициент детерминации) функция оценки регрессии.
from sklearn.metrics         import confusion_matrix, ConfusionMatrixDisplay, r2_score

# Вычислите рабочую характеристику приемника (ROC).
# Примечание: эта реализация ограничена задачей бинарной классификации.

# Вычислите площадь под кривой (AUC), используя правило трапеций.
from sklearn.metrics  import roc_curve, auc
from sklearn.datasets import make_classification

In [2]:
import MetaTrader5 as mt5 # treiding platforms
import talib       as tl # indocators

In [3]:
import pandas as pd # for datasets 
import numpy  as np # math for datasets

In [4]:
import matplotlib.pyplot as plt 
import seaborn           as sns
%matplotlib inline

In [5]:
from datetime import datetime, timedelta
import time

In [6]:
# интерактивные виджеты
# @interact
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [7]:
# Jupyter Notebook progressbar decorator
from tqdm import tqdm_notebook as tqdm

In [8]:
TAKE_PROFIT = 500
STOP_LOSS = 100

# Data  from the MetaTrader5

In [9]:
def conect_mt5():
    # Сonnect
    if mt5.initialize():
        login = 61715
        password = "b4980638"
        serwer = "GerchikCo-MT5"
        if mt5.login(login, password, serwer):
            ...
            print("Сonnected")
        else:
            print(f"{mt5.last_error()= }")

In [10]:
def my_tikers_from_mt5():
    tikers = """"*USD*,*EUR*,*JPY*,*GBP*,!*JPY*,!*SEK*,!*ZAR*,!*SEK*,!*NOK*,!*TRY*,!*MXN*,!*HKD*,!*DKK*,!*HUF*,!*PLN*,!*SGD*!*XAG*,!*XAU*,!*EUR_50*,!*XAG*"""
    conect_mt5()
    group_symbols = mt5.symbols_get(group = tikers)
    mt5.shutdown()
    name_tikers = [ s.name for s in group_symbols]
    return name_tikers

In [42]:
def data_mt5 (ticer = "EURUSDz", start_bar = 500,
              spacing = mt5.TIMEFRAME_H1, take_profit= 500,
             stop_loss =100 ):

    conect_mt5()
    
    data = mt5.copy_rates_from_pos(ticer, mt5.TIMEFRAME_H1, 0, start_bar)
    
    if mt5.shutdown():
        print("Shutdown")
    
    data_frame = pd.DataFrame(data)
    # преобразовать время в секундах в формат datetime
#     data_frame['time'] = pd.to_datetime( data_frame['time'], unit='s')
    
    # указываем метку которую в будущом будем предсказывать
    data_frame["positive_label"] = 0
    positive_label_query = ((data_frame["high"] - data_frame["open"] ) >= (take_profit/10**6))  &\
                            ((data_frame["open"] - data_frame["low"]) <= (stop_loss/10**5) )
    data_frame.loc[ positive_label_query, "positive_label" ] = 1
    
    data_frame["negative_label"] = 0
    negative_label_query = ((data_frame["open"] - data_frame["low"]) >= (take_profit/10**6)) &\
                            ((data_frame["high"] - data_frame["open"] ) <= (stop_loss/10**5) )  
    data_frame.loc[ negative_label_query, "negative_label" ] = 1

    return data_frame

In [35]:
def order(data_orders,
          direction = str,
          lot = 0.01 ,
          take_profit= 500,
          stop_loss = 100 ):
    """
    direction: In which direction should the buy or sell order be placed "BUY" or "SELL"
    """
    for symbol, value in data_orders.items():
        if value[-1] == 1:
            conect_mt5()
            symbol_info = mt5.symbol_info(symbol)
            if symbol_info is None:
                print(symbol, "not found, can not call order_check()")
                mt5.shutdown()
            # if the symbol is unavailable in MarketWatch, add it
            if not symbol_info.visible:
                print(symbol, "is not visible, trying to switch on")
                if not mt5.symbol_select(symbol,True):
                    print("symbol_select({}}) failed, exit",symbol)
                    mt5.shutdown()
        #     point = mt5.symbol_info(symbol).point
            request = dict()

            if direction == "BUY":
                print(direction, symbol )   
                request = {
                    "action": mt5.TRADE_ACTION_DEAL, 
                    "symbol": symbol,
                    "volume": lot,
                    "sl": mt5.symbol_info_tick(symbol).ask - stop_loss/10**5, 
                    "tp": mt5.symbol_info_tick(symbol).ask + take_profit/10**5,
                    "deviation": 10 ,
                    "type": mt5.ORDER_TYPE_BUY,
                    "type_filling": mt5.ORDER_FILLING_IOC ,
                    "magic": 234000,
                    "type_time": mt5.ORDER_TIME_DAY }

            elif direction == "SELL":
                print(direction, symbol ) 
                request={
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": lot,
                    "sl": mt5.symbol_info_tick(symbol).bid + stop_loss/10**5,
                    "tp": mt5.symbol_info_tick(symbol).bid - take_profit/10**5,
                    "deviation": 10,
                    "type": mt5.ORDER_TYPE_SELL, 
                    "type_filling": mt5.ORDER_FILLING_IOC ,
                    "magic": 234000,
                    "type_time": mt5.ORDER_TIME_DAY }

            mt5.order_send(request)
            mt5.shutdown()

# Indikator  from the Talib

In [13]:
def add_indicators_pattern_recognition_functions(data):
    df = data.copy()
    
    open = df["open"]
    high = df["high"]
    low = df["low"]
    close = df["close"]
    
    df["CDL2CROWS"] = tl.CDL2CROWS(open, high, low, close)
    df["CDL3BLACKCROWS"] = tl.CDL3BLACKCROWS(open, high, low, close)
    df["CDL3INSIDE"] = tl.CDL3INSIDE(open, high, low, close)
    df["CDL3LINESTRIKE"] = tl.CDL3LINESTRIKE(open, high, low, close)
    df["CDL3OUTSIDE"] = tl.CDL3OUTSIDE(open, high, low, close)
    df["CDL3STARSINSOUTH"] = tl.CDL3STARSINSOUTH(open, high, low, close)
    df["CDL3WHITESOLDIERS"] = tl.CDL3WHITESOLDIERS(open, high, low, close)
    df["CDLABANDONEDBABY"] = tl.CDLABANDONEDBABY(open, high, low, close, penetration=0)
    df["CDLADVANCEBLOCK"] = tl.CDLADVANCEBLOCK(open, high, low, close)
    df["CDLBELTHOLD"] = tl.CDLBELTHOLD(open, high, low, close)
    df["CDLBREAKAWAY"] = tl.CDLBREAKAWAY(open, high, low, close)
    df["CDLCLOSINGMARUBOZU"] = tl.CDLCLOSINGMARUBOZU(open, high, low, close)
    df["CDLCONCEALBABYSWALL"] = tl.CDLCONCEALBABYSWALL(open, high, low, close)
    df["CDLCOUNTERATTACK"] = tl.CDLCOUNTERATTACK(open, high, low, close)
    df["CDLDARKCLOUDCOVER"] = tl.CDLDARKCLOUDCOVER(open, high, low, close, penetration=0)
    df["CDLDOJI"] = tl.CDLDOJI(open, high, low, close)
    df["CDLDOJISTAR"] = tl.CDLDOJISTAR(open, high, low, close)
    df["CDLDRAGONFLYDOJI"] = tl.CDLDRAGONFLYDOJI(open, high, low, close)
    df["CDLENGULFING"] = tl.CDLENGULFING(open, high, low, close)
    df["CDLEVENINGDOJISTAR"] = tl.CDLEVENINGDOJISTAR(open, high, low, close, penetration=0)
    df["CDLEVENINGSTAR"] = tl.CDLEVENINGSTAR(open, high, low, close, penetration=0)
    df["CDLGAPSIDESIDEWHITE"] = tl.CDLGAPSIDESIDEWHITE(open, high, low, close)
    df["CDLGRAVESTONEDOJI"] = tl.CDLGRAVESTONEDOJI(open, high, low, close)
    df["CDLHAMMER"] = tl.CDLHAMMER(open, high, low, close)
    df["CDLHANGINGMAN"] = tl.CDLHANGINGMAN(open, high, low, close)
    df["CDLHARAMI"] = tl.CDLHARAMI(open, high, low, close)
    df["CDLHARAMICROSS"] = tl.CDLHARAMICROSS(open, high, low, close)
    df["CDLHIGHWAVE"] = tl.CDLHIGHWAVE(open, high, low, close)
    df["CDLHIKKAKE"] = tl.CDLHIKKAKE(open, high, low, close)
    df["CDLHIKKAKEMOD"] = tl.CDLHIKKAKEMOD(open, high, low, close)
    df["CDLHOMINGPIGEON"] = tl.CDLHOMINGPIGEON(open, high, low, close)
    df["CDLIDENTICAL3CROWS"] = tl.CDLIDENTICAL3CROWS(open, high, low, close)
    df["CDLINNECK"] = tl.CDLINNECK(open, high, low, close)
    df["CDLINVERTEDHAMMER"] = tl.CDLINVERTEDHAMMER(open, high, low, close)
    df["CDLKICKING"] = tl.CDLKICKING(open, high, low, close)
    df["CDLKICKINGBYLENGTH"] = tl.CDLKICKINGBYLENGTH(open, high, low, close)
    df["CDLLADDERBOTTOM"] = tl.CDLLADDERBOTTOM(open, high, low, close)
    df["CDLLONGLEGGEDDOJI"] = tl.CDLLONGLEGGEDDOJI(open, high, low, close)
    df["CDLLONGLINE"] = tl.CDLLONGLINE(open, high, low, close)
    df["CDLMARUBOZU"] = tl.CDLMARUBOZU(open, high, low, close)
    df["CDLMATCHINGLOW"] = tl.CDLMATCHINGLOW(open, high, low, close)
    df["CDLMATHOLD"] = tl.CDLMATHOLD(open, high, low, close, penetration=0)
    df["CDLMORNINGDOJISTAR"] = tl.CDLMORNINGDOJISTAR(open, high, low, close, penetration=0)
    df["CDLMORNINGSTAR"] = tl.CDLMORNINGSTAR(open, high, low, close, penetration=0)
    df["CDLONNECK"] = tl.CDLONNECK(open, high, low, close)
    df["CDLPIERCING"] = tl.CDLPIERCING(open, high, low, close)
    df["CDLRICKSHAWMAN"] = tl.CDLRICKSHAWMAN(open, high, low, close)
    df["CDLRISEFALL3METHODS"] = tl.CDLRISEFALL3METHODS(open, high, low, close)
    df["CDLSEPARATINGLINES"] = tl.CDLSEPARATINGLINES(open, high, low, close)
    df["CDLSHOOTINGSTAR"] = tl.CDLSHOOTINGSTAR(open, high, low, close)
    df["CDLSHORTLINE"] = tl.CDLSHORTLINE(open, high, low, close)
    df["CDLSPINNINGTOP"] = tl.CDLSPINNINGTOP(open, high, low, close)
    df["CDLSTALLEDPATTERN"] = tl.CDLSTALLEDPATTERN(open, high, low, close)
    df["CDLSTICKSANDWICH"] = tl.CDLSTICKSANDWICH(open, high, low, close)
    df["CDLTAKURI"] = tl.CDLTAKURI(open, high, low, close)
    df["CDLTASUKIGAP"] = tl.CDLTASUKIGAP(open, high, low, close)
    df["CDLTHRUSTING"] = tl.CDLTHRUSTING(open, high, low, close)
    df["CDLTRISTAR"] = tl.CDLTRISTAR(open, high, low, close)
    df["CDLUNIQUE3RIVER"] = tl.CDLUNIQUE3RIVER(open, high, low, close)
    df["CDLUPSIDEGAP2CROWS"] = tl.CDLUPSIDEGAP2CROWS(open, high, low, close)
    df["CDLXSIDEGAP3METHODS"] = tl.CDLXSIDEGAP3METHODS(open, high, low, close)
    
    return df.fillna(0)

In [14]:
def overlap_studies_functions(data, PERIOD = [23,115,460]):
    data = data.copy()
    close = data["close"]
    for i in PERIOD: 
        data["DEMA"+str(i)] = tl.DEMA(close, timeperiod=i)
        data["EMA"+str(i)] = tl.EMA(close, timeperiod=i)
        data["KAMA"+str(i)] = tl.KAMA(close, timeperiod=i)
        data["MIDPOINT"+str(i)] = tl.MIDPOINT(close, timeperiod=i)
        data["SMA"+str(i)] = tl.SMA(close, timeperiod=i)
        data["TRIMA"+str(i)] = tl.TRIMA(close, timeperiod=i)
        data["WMA"+str(i)] = tl.WMA(close, timeperiod=i)
        data["T3"+str(i)] = tl.T3(close, timeperiod=i, vfactor=0)
        data["TEMA"+str(i)] = tl.TEMA(close, timeperiod=i)
        data["MA"+str(i)] = tl.MA(close, timeperiod=i, matype=0)
    data["HT_TRENDLINE"+str(i)] = tl.HT_TRENDLINE(close)
    return data.fillna(0)

In [15]:
def math_transform_functions(data):
    df = data.copy()
    close = df["close"]
    
    df["ACOS"] = tl.ACOS(close)
    df["ASIN"] = tl.ASIN(close)
    df["ATAN"] = tl.ATAN(close)
    df["CEIL"] = tl.CEIL(close)
    df["COS"] =  tl.COS(close)
    df["COSH"] = tl.COSH(close)
    df["EXP"] =  tl.EXP(close)
    df["FLOOR"] = tl.FLOOR(close)
    df["LN"] =  tl.LN(close)
    df["LOG10"] = tl.LOG10(close)
    df["SIN"] =  tl.SIN(close)
    df["SINH"] = tl.SINH(close)
    df["SQRT"] = tl.SQRT(close)
    df["TAN"] =  tl.TAN(close)
    df["TANH"] = tl.TANH(close)
    
    return df.fillna(0)

In [16]:
def momentum_indicator_functions(data, PERIOD = [23,115,460]):
    df = data.copy()
    
    open = df["open"]
    high = df["high"]
    low = df["low"]
    close = df["close"]
    volume = df["real_volume"]
    for i in PERIOD:
        df["ADX"+str(i)] = tl.ADX(high, low, close, timeperiod=i)
        df["ADXR"+str(i)] = tl.ADXR(high, low, close, timeperiod=i)
        df["AROONOSC"+str(i)] = tl.AROONOSC(high, low, timeperiod=i)
        df["CCI"+str(i)] = tl.CCI(high, low, close, timeperiod=i)
        df["CMO"+str(i)] = tl.CMO(close, timeperiod=i)
        df["DX"+str(i)] = tl.DX(high, low, close, timeperiod=i)
        df["MFI"+str(i)] = tl.MFI(high, low, close, volume, timeperiod=i)
        df["MINUS_DI"+str(i)] = tl.MINUS_DI(high, low, close, timeperiod=i)
        df["MINUS_DM"+str(i)] = tl.MINUS_DM(high, low, timeperiod=i)
        df["MOM"+str(i)] = tl.MOM(close, timeperiod=i)
        df["PLUS_DI"+str(i)] = tl.PLUS_DI(high, low, close, timeperiod=i)
        df["PLUS_DM"+str(i)] = tl.PLUS_DM(high, low, timeperiod=i)
        df["ROC"+str(i)] =  tl.ROC(close, timeperiod=i)
        df["ROCP"+str(i)] =  tl.ROCP(close, timeperiod=i)
        df["ROCR"+str(i)] =  tl.ROCR(close, timeperiod=i)
        df["ROCR100"+str(i)] = tl.ROCR100(close, timeperiod=i)
        df["RSI"+str(i)] =   tl.RSI(close, timeperiod=i)
        df["WILLR"+str(i)] = tl.WILLR(high, low, close, timeperiod=i)
        df["TRIX"+str(i)] =  tl.TRIX(close, timeperiod=i)
    
    df["APO"] = tl.APO(close, fastperiod=12, slowperiod=26, matype=0)
    df["PPO"] = tl.PPO(close, fastperiod=12, slowperiod=26, matype=0)
    df["real"] = tl.ULTOSC(high, low, close, timeperiod1=7, timeperiod2=14, timeperiod3=28)
    df["real"] = tl.ULTOSC(high, low, close, timeperiod1=7, timeperiod2=14, timeperiod3=28)
    
#     df["macd","macdsignal","macdhistdf"]  = tl.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
#     df["MACDEXT","MACDEXTsignal","MACDEXThist"]  = tl.MACDEXT(close, fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
#     df["MACDFIX","MACDFIXsignal","MACDFIXhist"] = tl.MACDFIX(close, signalperiod=9)
#     df["STOCHRSIfastk","STOCHRSIfastd"] = tl.STOCHRSI(close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
#     df["STOCHslowk","STOCHslowd"] = tl.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
#     df["STOCHFfastk","STOCHFfastd"] = tl.STOCHF(high, low, close, fastk_period=5, fastd_period=3, fastd_matype=0)
    
    return df.fillna(0)

In [17]:
def statistic_functions (data, PERIOD = [23,115,460]):
    df = data.copy()
    high = df["high"]
    low = df["low"]
    close = df["close"]
    for i in PERIOD:
        df["BETA"+str(i)] = tl.BETA(high, low, timeperiod=i)
        df["CORREL"+str(i)] = tl.CORREL(high, low, timeperiod=i)
        df["LINEARREG"+str(i)] = tl.LINEARREG(close, timeperiod=i)
        df["LINEARREG_ANGLE"+str(i)] = tl.LINEARREG_ANGLE(close, timeperiod=i)
        df["LINEARREG_INTERCEPT"+str(i)] = tl.LINEARREG_INTERCEPT(close, timeperiod=i)
        df["LINEARREG_SLOPE"+str(i)] = tl.LINEARREG_SLOPE(close, timeperiod=i)
        df["STDDEV"+str(i)] = tl.STDDEV(close, timeperiod=i, nbdev=1)
        df["TSF"+str(i)] = tl.TSF(close, timeperiod=i)
        df["VAR"+str(i)] = tl.VAR(close, timeperiod=i, nbdev=1)
        
        df["median" + str(i)] = df["close"].rolling(window=i, min_periods=1).median()
        df["mode" + str(i)] = df["close"].rolling(window=i, min_periods=1).apply(lambda x: x.mode()[0])
        df["std" + str(i)] = df["median" + str(i)].rolling(window=i, min_periods=1).std()
        
    return df.fillna(0)

In [18]:
def math_operator_functions(data, PERIOD = [23,115,460]):
    df = data.copy()
    close = df["close"]
    high  = df["high"]
    low   = df["low"]
    
    for i in PERIOD:
        df["MAX"+str(i)] = tl.MAX(close, timeperiod=i)
        df["MAXINDEX"+str(i)] = tl.MAXINDEX(close, timeperiod=i)
        df["MIN"+str(i)] = tl.MIN(close, timeperiod=i)
        df["MININDEX"+str(i)] = tl.MININDEX(close, timeperiod=i)
        df["SUM"+str(i)] = tl.SUM(close, timeperiod=i)

# min, max = MINMAX(close, timeperiod=30)
# minidx, maxidx = MINMAXINDEX(close, timeperiod=30)

    df["ADD"] = tl.ADD(high, low)
    df["DIV"] = tl.DIV(high, low)
    df["SUB"] = tl.SUB(high, low)
    
    return df.fillna(0)

In [19]:
def best_indicators(data, PERIOD = [23,115,460] ) -> pd.DataFrame:
    
    """
    best indicators  zostały wybrane poprzez uczenie modelu na wskaźnikach
    o podobnym znaczeniu, takich jak wskaźniki „Overlap Studies” lub 
    „Momentum Indicators” i tak dalej… Wybórane zostali najbardziej przydatne dla modelu.
    """
    df = data.copy()
    open = df["open"]
    high = df["high"]
    low = df["low"]
    close = df["close"]
    
    
    df["CDLLONGLINE"] = tl.CDLLONGLINE(open, high, low, close)
    df["CDLSHORTLINE"] = tl.CDLSHORTLINE(open, high, low, close)
    df["CDLBELTHOLD"] = tl.CDLBELTHOLD(open, high, low, close)
    df["TANH"] = tl.TANH(close)
    df["ATAN"] = tl.ATAN(close)
    df["SINH"] = tl.SINH(close)
    df["DIV"] = tl.DIV(high, low)
    df["ADD"] = tl.ADD(high, low)
    df["SUB"] = tl.SUB(high, low)
    df["HT_TRENDLINE"] = tl.HT_TRENDLINE(close)
    df["WMA"] = tl.WMA(close, timeperiod=23)
    df["TRIMA"] = tl.TRIMA(close, timeperiod=23)
    df["CMO"] = tl.CMO(close, timeperiod=23)
    df["MINUS_DI"] = tl.MINUS_DI(high, low, close, timeperiod=23)
    df["ADX"] = tl.ADX(high, low, close, timeperiod=23)
    df["BETA"] = tl.BETA(high, low, timeperiod=23)
    df["TSF_23"] = tl.TSF(close, timeperiod=23)
    df["TSF_115"] = tl.TSF(close, timeperiod=115)
    df["mode_23"] = df["close"].rolling(window=23, min_periods=1).apply(lambda x: x.mode()[0])
    

    return df.fillna(0)

In [20]:
def best_parameters_for_buy (data) -> pd.DataFrame:
    """
    best indicators  zostały wybrane poprzez uczenie modelu na wskaźnikach
    o podobnym znaczeniu, takich jak wskaźniki „Overlap Studies” lub 
    „Momentum Indicators” i tak dalej… Wybórane zostali najbardziej przydatne dla modelu.
    13.01.2023
    """
    df = data.copy()
    open = df["open"]
    high = df["high"]
    low = df["low"]
    close = df["close"]
    
    df["CDLBELTHOLD"] = tl.CDLBELTHOLD(open, high, low, close)
    df["CDLMARUBOZU"] = tl.CDLMARUBOZU(open, high, low, close)
    df["CDLCLOSINGMARUBOZU"] = tl.CDLCLOSINGMARUBOZU(open, high, low, close)
    
    df["ATAN"] = tl.ATAN(close)
    df["COS"] =  tl.COS(close)
    df["COSH"] = tl.COSH(close)
    df["EXP"] =  tl.EXP(close)
    df["LN"] =  tl.LN(close)
    df["LOG10"] = tl.LOG10(close)
    df["SIN"] =  tl.SIN(close)
    df["SINH"] = tl.SINH(close)
    df["SQRT"] = tl.SQRT(close)
    df["TAN"] =  tl.TAN(close)
    df["TANH"] = tl.TANH(close)
    
    return df.fillna(0)

In [21]:
def best_parameters_for_sell(data):
    """
    best parameters
    """
    df = data.copy()
    
    open = df["open"]
    high = df["high"]
    low = df["low"]
    close = df["close"]
    
    
    df["CDLLONGLINE"] = tl.CDLLONGLINE(open, high, low, close)
    df["CDLSHORTLINE"] = tl.CDLSHORTLINE(open, high, low, close)
    df["CDLBELTHOLD"] = tl.CDLBELTHOLD(open, high, low, close)
    df["CDLENGULFING"] = tl.CDLENGULFING(open, high, low, close)
    
    df["WILLR_23"] = tl.WILLR(high, low, close, timeperiod=23)
    df["APO"] = tl.APO(close, fastperiod=12, slowperiod=26, matype=0)
    df["PPO"] = tl.PPO(close, fastperiod=12, slowperiod=26, matype=0)
    
    return df.fillna(0)

# Prepared models and data for forecasting

In [22]:
def all_data_for_predict(tiker, indicators, start_bar=500 ) -> dict :
    """
    "BUY" or "SELL"
    all data from MT5 for predict
    wszystkie dane z MT5 do przewidywania 
    """
    data = dict()
    for i in tiker:
        data[str(i)] = indicators( data_mt5( ticer=i, start_bar=start_bar))
    return data
        
data_for_buy = all_data_for_predict( my_tikers_from_mt5(), best_parameters_for_buy, start_bar=1 )
data_for_sell = all_data_for_predict( my_tikers_from_mt5(), best_parameters_for_sell, start_bar=1 )

Сonnected
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown


In [23]:
%%time
def model_training (data ,direction= str, return_test_data = True): 
    """
    data: model training data
    direction:"str" the direction in which transactions will be carried out to choose from "BUY" or "SELL"
    return: If return_test_data = True model -> best_clf, test data -> X_test and y_test
            else return only model -> best_clf
    """
    
    if direction == "BUY": predict_label = data.positive_label
    elif direction == "SELL" : predict_label = data.negative_label
    
    #направление сделки
    X = data.drop(["negative_label","positive_label","time",
                   "real_volume","high","low","close","spread"], axis=1)
    y = predict_label
        
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        test_size=0.33, 
                                                        random_state=42) 
    
     # RandomForestClassifier — это метаоценка, которая соответствует ряду классификаторов дерева 
     # решений для различных подвыборок набора данных и использует усреднение для повышения 
     # точности прогнозирования и контроля переобучения.
    clf_rf = RandomForestClassifier(random_state=0)
    params = ({ "criterion": ["gini", "entropy", "log_loss" ],
                "n_estimators":range(10,200,10),  # от 10 до 100 с шагом 10
                "max_depth": range(3,7,2),        # от 3 до 7 с шагом 2
                "min_samples_leaf":range(1,7),    # от 1 до 7
                "min_samples_split":range(2,9,2)  # от 2 до 9 с шагом 2
              })

    # GridSearchCV реализует метод «подгонки» и «оценки».
    grid_searh_cv_clf = GridSearchCV(clf_rf, 
                                     params,
                                     cv= 5,
                                     n_jobs=-1,
                                     error_score="raise" ) 

    # обучение модели
    grid_searh_cv_clf.fit(X_train, y_train)

    # выбор самых лутших параметров ссовсего леса
    best_clf = grid_searh_cv_clf.best_estimator_
    
    if return_test_data == True : return [best_clf, X_test, y_test]
    else: return [best_clf ]
    
model_for_buy  =  model_training( best_parameters_for_buy( data_mt5()), direction="BUY" )
model_for_sell = model_training( best_parameters_for_sell( data_mt5()), direction="SELL" )
# Wall time: 48min 34s

Сonnected
Shutdown
Сonnected
Shutdown
CPU times: total: 1min 35s
Wall time: 44min 49s


In [24]:
def predict_data( data_for_predicted  , model, percent = 0.5 ) -> int :
    """
    data_for_predicted: model for predicts
    model: model for predicts
    percent: recommendation = 0.75, 0.8 because "recall" approaching zero
    resust: prediction in int form 
    """
    if type(model) == list:
        model = model[0]
    data_for_predicted = data_for_predicted.drop(["negative_label","positive_label","time",
                   "real_volume","high","low","close","spread"], axis=1 ) 
    predict = model.predict(data_for_predicted)
    predict_pr = model.predict_proba(data_for_predicted)
    predict_result = np.where( predict_pr[:, 1] > percent , 1, 0 )
    return  predict_result

In [25]:
def all_predict_data (models, datas, percents = 0.5 ) -> dict :
    """
    datas: all data for predicts
    models: all model for predicts
    resust: prediction in the form of an dict which has the form of a key  ticker and volume predict
    """
    predict = dict()
    for key, value in datas.items():
        predict[key] = predict_data(data_for_predicted = datas[key], 
                                    model = models, percent = percents )
    return predict

In [26]:
# percentage_experiment_and_informational
def model_information( model, 
                      X = False , 
                      y = False, percent = 0.5,
                      info = False, visualization = False ):
    """
    model: model predict 
    X -> X_test
    y -> y_test
    
    """    
    if type(model) == list :
        model, X_test, y_test = model
    elif type(model) != list :
        X_test = X
        y_test = y
    # predict data
    predicts = model.predict(X_test)
    
    # Метод в классификаторах и кластеризаторах, который может возвращать оценки вероятности для каждого класса/кластера. 
    # На его вход обычно поступают только наблюдаемые данные X.
    test_predict_proba = model.predict_proba(X_test)
    pred_prob = np.where( test_predict_proba[:, 1] > percent , 1, 0 )

    # Precision — полезная мера успеха предсказания, когда классы очень несбалансированы.
    # В информационном поиске точность — это мера релевантности результатов. 
    precision = precision_score(y_test, pred_prob, average='micro')
    
    # recall — это мера того, сколько действительно релевантных результатов возвращается.
    recall = recall_score(y_test, pred_prob, average='micro')
    
    # метрика насколько точно мы предсказываем правильные знаения как дя 1 так и для 0 
    accuracy_scor_before_setting = accuracy_score(y_test, predicts)
    accuracy_scor_after_setting = accuracy_score(y_test, pred_prob)
    
    
    if info == True :
        print(f"before change = {accuracy_scor_before_setting}")
        print(f"after change  = {accuracy_scor_after_setting}")
        print(f"{precision = }\n{recall = }")
    elif visualization == True:
        pd.Series(pred_prob).hist()
        
        # указывае относительную меру важности каждого признака (фичи) в предсказаниях модели
        feature_importances = model.feature_importances_
        feature_importances_df = pd.DataFrame({"feature": X_test.columns,
                                               "feature_importances":feature_importances})\
                                                .sort_values("feature_importances", 
                                                             ascending=False)

        # визуализация относительных мер важности каждого признака(фичи) в предсказаниях модели
        f, ax = plt.subplots(figsize=(6, 6))
        sns.barplot(y=feature_importances_df.feature, 
                    x= feature_importances_df.feature_importances)
        plt.xlabel('Важность атрибутов')
        plt.ylabel('Атрибуты')
        plt.title("Важность атрибутыов")
        plt.show()
        
        
        print("""        
                истинно положительный (TP)
                Результат теста, который правильно указывает на наличие условия или характеристики
                истинно отрицательный (TN)
                Результат теста, который правильно указывает на отсутствие условия или характеристики
                ложноположительный результат (FP)
                Результат теста, который ошибочно указывает на наличие определенного состояния или атрибута
                ложноотрицательный (FN)
                Результат теста, ошибочно указывающий на отсутствие определенного условия или атрибута.
                
                Оши́бка пе́рвого ро́да (α-ошибка,ложноположительное заключение) — ситуация, когда 
                отвергнута верная нулевая гипотеза (об отсутствии связи между явлениями или искомого эффекта)
                Оши́бка второ́го ро́да (β-ошибка, ложноотрицательное заключение) — 
                ситуация, когда принята неверная нулевая гипотеза.
                """)
        cm = confusion_matrix(y_test, pred_prob, labels=model.classes_)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                      display_labels=model.classes_)
        disp.plot()
        plt.show()
        
        # Визуализация ROC AUC кривой 
        # показатель роспределения наших предсказаний по precision и recall
        fpr, tpr, thresholds = roc_curve(y_test, pred_prob)
        roc_auc= auc(fpr, tpr)

        plt.figure()
        plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % (roc_auc))
        plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
        plt.axis("square")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)")
        plt.legend()
        return plt.show()

In [27]:
@interact
def fo(b=(0.5,1,0.1), s=(0.5,1,0.1)):
    model_information(model_for_buy,info=True, percent=b)
    model_information(model_for_buy,visualization=True, percent=b)

    model_information(model_for_sell,info=True, percent=s)
    model_information(model_for_sell,visualization=True, percent=s)

interactive(children=(FloatSlider(value=0.7, description='b', max=1.0, min=0.5), FloatSlider(value=0.7, descri…

# Predicting prices

In [28]:
def mein():
    while True :
        data_for_buy = all_data_for_predict( my_tikers_from_mt5(), 
                                                best_parameters_for_buy, 
                                                start_bar=1000 )
        data_for_sell = all_data_for_predict( my_tikers_from_mt5(),
                                                 best_parameters_for_sell, 
                                                 start_bar=1000 )
            
        buy_predict  =  all_predict_data(models= model_for_buy,
                             datas= data_for_buy )
        sell_predict =  all_predict_data(models= model_for_sell,
                                 datas= data_for_sell)

        order(data_orders=buy_predict, direction= "BUY")
        order(data_orders=sell_predict, direction= "SELL")
        time.sleep(60*60)
        

def mein_2(): 
    hour_control = 0 
    while True :
        now = datetime.now()
        if now.hour != hour_control:
            hour_control = now.hour
            
            data_for_buy = all_data_for_predict( my_tikers_from_mt5(), 
                                                best_parameters_for_buy, 
                                                start_bar=1000 )
            data_for_sell = all_data_for_predict( my_tikers_from_mt5(),
                                                 best_parameters_for_sell, 
                                                 start_bar=1000 )
            
            buy_predict  =  all_predict_data(models= model_for_buy,
                                             datas= data_for_buy,
                                             percents = 0.6)
            sell_predict =  all_predict_data(models= model_for_sell,
                                             datas= data_for_sell,
                                             percents = 0.6 )
            

            order(data_orders=buy_predict, direction= "BUY")
            order(data_orders=sell_predict, direction= "SELL")

In [44]:
if __name__ == "__main__":
    mein_2() 

Сonnected
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
BUY EURCHFz
Сonnected
BUY EURNZDz
Сonnected
BUY GBPCHFz
Сonnected
BUY GBPNZDz
Сonnected
SELL EURAUDz
Сonnected
SELL EURNZDz
Сonnected
SELL GBPNZDz
Сonnected
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Сonnected
Shutdown
Сonnected
Shutdown
Сonnected
Shutdown
Сonne

KeyError: 'high'

# first experyments

# Choosing the best indicators for our model

In [ ]:
# %pdb
df = data_mt5(start_bar=2000)

In [ ]:
data_pattern           = add_indicators_pattern_recognition_functions(df)
data_overlap           = overlap_studies_functions(df)
data_momentum          = momentum_indicator_functions(df)
data_statistic         = statistic_functions(df)
data_math_transform    = math_transform_functions(df)
data_math_operator     = math_operator_functions(df)

In [ ]:
%%time

model_data_pattern        = model_training(data_pattern,"BUY")         # 01 - 35 | 11 - 184
model_data_math_transform = model_training(data_math_transform,"BUY")  # 01 - 92 | 11 - 149

model_data_overlap        = model_training(data_overlap,"BUY")
model_data_momentum       = model_training(data_momentum,"BUY")
model_data_statistic      = model_training(data_statistic,"BUY")
model_data_math_operator  = model_training(data_math_operator,"BUY")
# Wall time: 3h 43min 1s

In [ ]:
@interact
def foo(tiker= my_tikers_from_mt5(), 
        percent= (0.5,1,0.1),
        models= [model_data_math_operator,
                 model_data_pattern,        
                 model_data_math_transform, 
                 model_data_overlap,        
                 model_data_momentum,       
                 model_data_statistic]):
    model_information(model= model_data_math_transform, percent = percent, info=True )    
    model_information(model= model_data_overlap, percent =percent, visualization=True,)

### Testowanie "najlepszych" modelej na nowych danych

In [ ]:
@interact
def foo(tiker= my_tikers_from_mt5(), percent= (0.5,1,0.1)):
    # test data that the model has not previously seen 
    df_test = data_mt5(tiker , start_bar=5000)
    df_test_pattern_recognition_ = add_indicators_pattern_recognition_functions(df_test)
    df_test_pattern_recognition_ = df_test_pattern_recognition_.head(1000)
    y_df_test_pattern_recognition_ = df_test_pattern_recognition_.positive_label
    X_df_test_pattern_recognition_ = df_test_pattern_recognition_.drop(["negative_label","positive_label","time",
                       "real_volume","high","low","close","spread"], axis=1)
    print(y_df_test_pattern_recognition_.sum())
    
    additional_information_about_the_model(predict_model=model_data_pattern ,
                                      X_test= X_df_test_pattern_recognition_, 
                                      y_test= y_df_test_pattern_recognition_,
                                      percent = percent, visualization=True)


In [ ]:
@interact
def foo(tiker= my_tikers_from_mt5(), percent= (0,1,0.1)):
    # test data that the model has not previously seen 
    df_test = data_mt5(start_bar=5000)
    df_test_math_transform = math_transform_functions(df_test)
    df_test_math_transform = df_test_math_transform.head(1000)
    y_df_test_math_transform = df_test_math_transform.positive_label
    X_df_test_math_transform = df_test_math_transform.drop(["negative_label","positive_label","time",
                       "real_volume","high","low","close","spread"], axis=1)
    print(y_df_test_math_transform.sum())
    
    additional_information_about_the_model(predict_model=model_data_math_transform[0] ,
                                      X_test= X_df_test_math_transform, 
                                      y_test= y_df_test_math_transform,
                                      percent = percent, visualization=True)

# Testing the best parameters of our models
# Testujemy najlepsze parametry naszych modeli

In [ ]:
def pattern_and_math_transform(data) -> pd.DataFrame:
    """
    best indicators  zostały wybrane poprzez uczenie modelu na wskaźnikach
    o podobnym znaczeniu, takich jak wskaźniki „Overlap Studies” lub 
    „Momentum Indicators” i tak dalej… Wybórane zostali najbardziej przydatne dla modelu.
    """
    df = data.copy()
    open = df["open"]
    high = df["high"]
    low = df["low"]
    close = df["close"]
    
    df["CDLDARKCLOUDCOVER"] = tl.CDLDARKCLOUDCOVER(open, high, low, close, penetration=0)
    df["CDLBELTHOLD"] = tl.CDLBELTHOLD(open, high, low, close)
    df["CDLMARUBOZU"] = tl.CDLMARUBOZU(open, high, low, close)
    df["CDL3INSIDE"] = tl.CDL3INSIDE(open, high, low, close)
    df["CDLCLOSINGMARUBOZU"] = tl.CDLCLOSINGMARUBOZU(open, high, low, close)
    
    df["ACOS"] = tl.ACOS(close)
    df["ASIN"] = tl.ASIN(close)
    df["ATAN"] = tl.ATAN(close)
    df["COS"] =  tl.COS(close)
    df["COSH"] = tl.COSH(close)
    df["EXP"] =  tl.EXP(close)
    df["LN"] =  tl.LN(close)
    df["LOG10"] = tl.LOG10(close)
    df["SIN"] =  tl.SIN(close)
    df["SINH"] = tl.SINH(close)
    df["SQRT"] = tl.SQRT(close)
    df["TAN"] =  tl.TAN(close)
    df["TANH"] = tl.TANH(close)
    
    return df.fillna(0)

In [ ]:
%%time
df = data_mt5(start_bar=2000)
data_best_parameters = pattern_and_math_transform(df)
model_best_parameters = model_training(data_best_parameters,direction="BUY")
# Wall time: 38min 5s

In [ ]:
percentage_for_better_selection(model= model_best_parameters, percent = 0.5, info=True )

In [ ]:
percentage_for_better_selection(model= model_best_parameters, percent = 0.5, visualization=True,)

In [ ]:
df_test = data_mt5(start_bar=5000)
df_test_best_parameters = pattern_and_math_transform(df_test)
df_test_best_parameters = df_test_best_parameters.head(1000)
y_df_test_best_parameters = df_test_best_parameters.positive_label
X_df_test_best_parameters = df_test_best_parameters.drop(["negative_label","positive_label","time",
                   "real_volume","high","low","close","spread"], axis=1)
y_df_test_best_parameters.sum()

In [ ]:
additional_information_about_the_model(predict_model=model_best_parameters[0] ,
                                      X_test= X_df_test_best_parameters, 
                                      y_test= y_df_test_best_parameters,
                                      percent = 0.5, visualization=True)

# refinement of the model by removing redundant data
# udoskonalenie modelu poprzez usunięcie zbędnych danych

In [ ]:
def pattern_and_math_transform_2(data) -> pd.DataFrame:
    """
    best indicators  zostały wybrane poprzez uczenie modelu na wskaźnikach
    o podobnym znaczeniu, takich jak wskaźniki „Overlap Studies” lub 
    „Momentum Indicators” i tak dalej… Wybórane zostali najbardziej przydatne dla modelu.
    """
    df = data.copy()
    open = df["open"]
    high = df["high"]
    low = df["low"]
    close = df["close"]
    
    df["CDLBELTHOLD"] = tl.CDLBELTHOLD(open, high, low, close)
    df["CDLMARUBOZU"] = tl.CDLMARUBOZU(open, high, low, close)
    df["CDLCLOSINGMARUBOZU"] = tl.CDLCLOSINGMARUBOZU(open, high, low, close)
    
    df["ATAN"] = tl.ATAN(close)
    df["COS"] =  tl.COS(close)
    df["COSH"] = tl.COSH(close)
    df["EXP"] =  tl.EXP(close)
    df["LN"] =  tl.LN(close)
    df["LOG10"] = tl.LOG10(close)
    df["SIN"] =  tl.SIN(close)
    df["SINH"] = tl.SINH(close)
    df["SQRT"] = tl.SQRT(close)
    df["TAN"] =  tl.TAN(close)
    df["TANH"] = tl.TANH(close)
    
    return df.fillna(0)

In [ ]:
%%time
df = data_mt5(start_bar=2000)
data_best_parameters_2 = pattern_and_math_transform_2(df)
model_best_parameters_2 = model_training(data_best_parameters_2,direction="BUY")
# Wall time: 38min 5s

In [ ]:
@interact
def foo(tiker= my_tikers_from_mt5(),percent=(0.5,1,0.1)):
    df_test = data_mt5(ticer='GBPCADz', start_bar=5000,)
    df_test_best_parameters_2 = pattern_and_math_transform_2(df_test)
    df_test_best_parameters_2 = df_test_best_parameters_2.head(115)
    y_df_test_best_parameters_2 = df_test_best_parameters_2.positive_label
    X_df_test_best_parameters_2 = df_test_best_parameters_2.drop(["negative_label","positive_label","time",
                       "real_volume","high","low","close","spread"], axis=1)
    print(y_df_test_best_parameters_2.sum())

    additional_information_about_the_model(predict_model=model_best_parameters_2[0] ,
                                          X_test= X_df_test_best_parameters_2, 
                                          y_test= y_df_test_best_parameters_2,
                                          percent = percent, visualization=True)
    additional_information_about_the_model(predict_model=model_best_parameters_2[0] ,
                                      X_test= X_df_test_best_parameters_2, 
                                      y_test= y_df_test_best_parameters_2,
                                      percent = percent, info=True)


# test for sell
# test na  sell

In [ ]:
model_best_parameters_2_sell = model_training(data_best_parameters_2,direction="SELL")

In [ ]:
@interact
def foo(tiker= my_tikers_from_mt5(), percent=(0.5,1,0.1)):
    df_test = data_mt5(start_bar=5000)
    df_test_best_parameters_2_sell = pattern_and_math_transform_2(df_test)
    df_test_best_parameters_2_sell = df_test_best_parameters_2_sell.head(1000)
    y_df_test_best_parameters_2_sell = df_test_best_parameters_2_sell.negative_label 
    X_df_test_best_parameters_2_sell = df_test_best_parameters_2_sell.drop(["negative_label","positive_label","time",
                       "real_volume","high","low","close","spread"], axis=1)
    print(y_df_test_best_parameters_2_sell.sum())

    additional_information_about_the_model(predict_model=model_best_parameters_2_sell[0] ,
                                          X_test= X_df_test_best_parameters_2, 
                                          y_test= y_df_test_best_parameters_2,
                                          percent =percent, info=True)
    additional_information_about_the_model(predict_model=model_best_parameters_2_sell[0] ,
                                          X_test= X_df_test_best_parameters_2, 
                                          y_test= y_df_test_best_parameters_2,
                                          percent = percent, visualization=True)


#### tego typu wskaźniki do przewidywania ceny w KUPUJ "BUY" wydawały się być bardzo dobre, ale w sprzedaży "SELL" jest bardzo źle dla tego nazweme funkcje buy_indikators i będe ona uzywana wysłaćnie do KUPNO.

# Choosing the best indicators SELL for our model
# Wybór najlepszych wskaźników SPRZEDAJ dla naszego modelu

In [ ]:
df = data_mt5(start_bar=2000)

In [ ]:
data_pattern           = add_indicators_pattern_recognition_functions(df)
data_overlap           = overlap_studies_functions(df)
data_momentum          = momentum_indicator_functions(df)
data_statistic         = statistic_functions(df)
data_math_transform    = math_transform_functions(df)
data_math_operator     = math_operator_functions(df)

In [ ]:
%%time

model_data_pattern        = model_training(data_pattern,"SELL")        
model_data_math_transform = model_training(data_math_transform,"SELL") 
model_data_overlap        = model_training(data_overlap,"SELL")
model_data_momentum       = model_training(data_momentum,"SELL")
model_data_statistic      = model_training(data_statistic,"SELL")
model_data_math_operator  = model_training(data_math_operator,"SELL")
# Wall time: 3h 43min 1s

In [ ]:
# search best parameters
@interact
def foo( percents= (0.5,1,0.1),
         model= [ model_data_pattern,     # 81% 01 - 24 | 11 - 185
                 model_data_momentum,      # 62% 01 - 96 | 11 - 129
                 model_data_math_operator,      
                 model_data_math_transform, 
                 model_data_overlap,        
                 model_data_statistic]):
    
    model_information(models= model, percent = percents, info=True )    
    model_information(models= model, percent =percents, visualization=True,)

In [ ]:
def pattern_and_momentum(data):
    """
    best parameters
    """
    df = data.copy()
    
    open = df["open"]
    high = df["high"]
    low = df["low"]
    close = df["close"]
    
    
    df["CDLLONGLINE"] = tl.CDLLONGLINE(open, high, low, close)
    df["CDLSHORTLINE"] = tl.CDLSHORTLINE(open, high, low, close)
    df["CDLBELTHOLD"] = tl.CDLBELTHOLD(open, high, low, close)
    df["CDLENGULFING"] = tl.CDLENGULFING(open, high, low, close)
    
    df["WILLR_23"] = tl.WILLR(high, low, close, timeperiod=23)
    df["APO"] = tl.APO(close, fastperiod=12, slowperiod=26, matype=0)
    df["PPO"] = tl.PPO(close, fastperiod=12, slowperiod=26, matype=0)
    
    return df.fillna(0)

In [ ]:
%%time
# model for best parameters
df = data_mt5(start_bar=2000)
data_best_parameters = pattern_and_momentum(df)
model_best_parameters = model_training(data_best_parameters,direction="SELL")

In [ ]:
# result which best parameters
@interact
def foo(tiker= my_tikers_from_mt5(),percent=(0.5,1,0.1)):
    df_test = data_mt5(ticer=tiker, start_bar=5000,)
    df_test_best_parameters = pattern_and_momentum(df_test)
    df_test_best_parameters = df_test_best_parameters.head(115)
    y_df_test_best_parameters = df_test_best_parameters.negative_label
    X_df_test_best_parameters = df_test_best_parameters.drop(["negative_label","positive_label","time",
                       "real_volume","high","low","close","spread"], axis=1)
    print(y_df_test_best_parameters.sum())

    model_information(model=model_best_parameters[0] ,
                                          X= X_df_test_best_parameters, 
                                          y= y_df_test_best_parameters,
                                          percent = percent, visualization=True)
    model_information(model=model_best_parameters[0] ,
                                      X= X_df_test_best_parameters, 
                                      y= y_df_test_best_parameters,
                                      percent = percent, info=True)

# second experiments

# data preparation 

In [ ]:
def data_from_all_indikators(df):
    data_pattern           = add_indicators_pattern_recognition_functions(df)
    data_overlap           = overlap_studies_functions(data_pattern)
    data_momentum          = momentum_indicator_functions(data_overlap)
    data_statistic         = statistic_functions(data_momentum)
    data_math_transform    = math_transform_functions(data_statistic)
    all_indicators     = math_operator_functions(data_math_transform)
    return all_indicators

In [ ]:
# проверяем кореляцию
def correlations(data, lebels = "positive_label", kind_of_correlation = "pozitiv",
                name_indicators = False, perc_pos_cor = 0,
                perc_neg_cor = 0):
    """
    data: pandas Data Fream
    kind_of_correlation: pozitiv corelation or negativ kokrelation
    lebels: "positive_label" or "negative_label"
    name_indicators: return indikators name 
    perc_pos_cor: percent correlations 
    perc_pos_cor: percent correlations 
    """
    corelation_indikators = dict()
    for i in data.select_dtypes('number').columns:
        corelation_indikators[i] = data[lebels].corr(data[i])

    cor_indic = pd.DataFrame({"indicators": corelation_indikators.keys(),
                              "correlation": corelation_indikators.values()})
    
    if lebels == "positive_label":
        positiv_cor_indicatorrs = cor_indic[cor_indic["correlation"]  > perc_pos_cor ]
        negaitiv_cor_indicatorrs = cor_indic[cor_indic["correlation"] < -perc_neg_cor ]
        
    elif lebels == "negative_label":
        positiv_cor_indicatorrs = cor_indic[cor_indic["correlation"]  > perc_pos_cor ]
        negaitiv_cor_indicatorrs = cor_indic[cor_indic["correlation"] < -perc_neg_cor ]
    
    indicators = lambda x: list(x["indicators"])
    
    if kind_of_correlation == "pozitiv":
        if name_indicators:
            return indicators(positiv_cor_indicatorrs)
        return positiv_cor_indicatorrs
    
    elif kind_of_correlation == "negativ":
        if name_indicators:
            return indicators(negaitiv_cor_indicatorrs)
        return negaitiv_cor_indicatorrs

In [ ]:
def correlation_data(all_data_indicators = pd.DataFrame, name_indicators = list ):
    df = pd.DataFrame()
    for i in name_indicators:
        df[i] =  all_data_indicators[i]
    return df


In [ ]:
df = data_mt5(start_bar=20000)

In [ ]:
pos_lab_cor =  correlations(data_from_all_indikators(df), 
                            "positive_label", name_indicators=True, perc_pos_cor=0.1 )
df_buy  = correlation_data(all_indicators, pos_lab_cor)

In [ ]:
test_data = data_from_all_indikators(data_mt5("EURAUDz",start_bar=10000))
test_data = correlation_data(test_data, pos_lab_cor)

In [ ]:
# neg_lab_cor =  correlations(all_indicators, "negative_label", name_indicators=True, perc_pos_cor=0.08)
# df_sell = correlation_data(all_indicators, neg_lab_cor)

# CatBoostClassifier

# model preparationm

# 0.74

In [ ]:
from catboost import CatBoostClassifier, Pool, metrics, cv

In [ ]:
CatBoostClassifier()

In [ ]:
X = df_buy.drop('positive_label', axis=1)
y = df_buy.positive_label

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.32, random_state=42)

In [ ]:
model_catboostcl = CatBoostClassifier(
    custom_loss=[metrics.AUC()],
    random_seed=42 )

In [ ]:
model_catboostcl.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
#     logging_level='Verbose',  # you can uncomment this for text output );

In [ ]:
cv_params = {
    "learning_rate":0.1,
    "eval_metric":'AUC',
#     "cusstom_loss":['AUC', 'Accuracy'],
    "loss_function":"Logloss",
    "random_seed":42,
    "early_stopping_rounds":20,
    'loss_function': metrics.Logloss()}

cv_data = cv(
    params= cv_params,
    pool=Pool(X, y ),
    fold_count = 5,
    shuffle= False,
    plot=False )
cv()

In [ ]:
for name, values in cv_data.iteritems():
    print(name + ':')
    print(np.array(values))
    print('\n')

In [ ]:
best_value = np.max(cv_data['test-AUC-mean'])
best_iter = np.argmax(cv_data['test-AUC-mean'])
best_iter

# model predict

In [ ]:
y = test_data.positive_label
x = test_data.drop("positive_label",axis=1)
pool = Pool(X,y)

In [ ]:
tree_count = model.tree_count_
metrics = model.eval_metrics(pool, metrics=['Logloss','AUC','Accuracy'], ntree_start=0, ntree_end=0,
                             eval_period=tree_count, thread_count=2)
auc = metrics['AUC']
print(auc)